<a href="https://colab.research.google.com/github/humblefool1997/nnlj/blob/main/NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Load Data
*   Define PyToch Model
*   Define Loss Function and Optimizers
*   Run a Training Loop
*   Evaluate the Model
*   Make Predictions



In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import urllib.request


In [12]:
# Load data

url  = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

download_path = "data.csv"

# Download the file
urllib.request.urlretrieve(url, download_path)

# Load the file using np.loadtxt
data = np.loadtxt('data.csv',delimiter=',')

In [13]:
X = data[:,0:8]
Y = data[:,8]
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y,dtype=torch.float32).reshape(-1,1)

# define the model
class FOCAST(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden1 = nn.Linear(8,12)
    self.act1 = nn.ReLU()
    self.hidden2 = nn.Linear(12,8)
    self.act2 = nn.ReLU()
    self.output = nn.Linear(8,1)
    self.act_output = nn.Sigmoid()
  def forward(self,x):
    x = self.act1(self.hidden1(x))
    x = self.act2(self.hidden2(x))
    x = self.act_output(self.output(x))
    return x

model = FOCAST()

print(model)


FOCAST(
  (hidden1): Linear(in_features=8, out_features=12, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=12, out_features=8, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
  (act_output): Sigmoid()
)


In [14]:
# training the model

loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

number_of_epochs = 100
batch_size = 10

for epoch in range(number_of_epochs):
    for i in range(0, len(X), batch_size):
        XBatch = X[i:i+batch_size]
        YBatch = Y[i:i+batch_size]
        YPrediction = model(XBatch)
        YPrediction = YPrediction.view(YBatch.size())  # Reshape output tensor if needed
        loss = loss_fn(YPrediction, YBatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

YPred = model(X)
accuracy = (YPred.round() == Y).float().mean()
print(f"Accuracy: {accuracy}")

# Make class predictions with the model
predictions = (model(X) > 0.5).int()
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], Y[i]))

Accuracy: 0.7447916865348816
[6.0, 148.0, 72.0, 35.0, 0.0, 33.599998474121094, 0.6269999742507935, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.600000381469727, 0.35100001096725464, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.299999237060547, 0.671999990940094, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.100000381469727, 0.16699999570846558, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.099998474121094, 2.2880001068115234, 33.0] => 1 (expected 1)


In [26]:
!pip3 install onnx onnxscript onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.3 MB/s eta 0:00:00


In [23]:
torch_model = FOCAST()
torch_input = torch.randn(1, 1, 12, 8)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [24]:
onnx_program.save("FOCAST_NN.onnx")

In [27]:
import onnxruntime

onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./FOCAST_NN.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

Input length: 1
Sample input: (tensor([[[[-1.0975, -0.2460, -0.8397, -0.0299, -0.4566, -0.5399,  0.2287,
            1.1354],
          [ 0.2854, -1.6507,  1.3029, -0.9485, -0.3040,  0.5208, -1.4332,
            0.1326],
          [-0.4551, -2.6082, -0.4451,  0.3021, -2.0437,  0.4382, -0.9650,
           -1.1040],
          [-2.0635, -0.9618,  0.5340,  2.1871,  0.9119, -0.8013, -0.8848,
            0.1226],
          [ 0.5032,  1.4424,  0.8510,  0.2507, -1.6922,  0.6560, -0.9740,
            1.5166],
          [ 0.6723, -0.2180,  0.9865, -0.9899,  2.0724,  1.9553,  0.7595,
           -1.7206],
          [-0.8144,  0.0927, -2.0373, -0.6052,  0.3486, -0.5206, -0.3024,
            3.3848],
          [-0.4549, -0.7630, -0.8956, -0.8808, -0.3955,  1.6807,  0.6848,
           -0.2610],
          [ 1.3772,  0.6872, -1.0175,  0.9090, -0.8344,  1.0668, -0.7011,
            0.6986],
          [ 0.0605,  0.6856,  2.5146,  1.8676,  0.6236, -0.0886,  0.0898,
            0.1780],
          [ 0.0412,

In [28]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_program.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[[[0.55346644],
         [0.63283217],
         [0.6099308 ],
         [0.53460914],
         [0.5578151 ],
         [0.60101694],
         [0.5393855 ],
         [0.58695143],
         [0.5622459 ],
         [0.5508343 ],
         [0.5815853 ],
         [0.56927437]]]], dtype=float32)]
